In [89]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn as sk
import pickle
import copy
from pyts.classification import TimeSeriesForest
from sklearn.model_selection import train_test_split


In [2]:
AU_COLS = ['AU01_r',
'AU02_r',
'AU04_r',
'AU05_r',
'AU06_r',
'AU07_r',
'AU09_r',
'AU10_r',
'AU12_r',
'AU14_r',
'AU15_r',
'AU17_r',
'AU20_r',
'AU23_r',
'AU25_r',
'AU26_r',
'AU45_r']

ID_COLS = ['filename', "frame", "video_id", "emotion_1_id"]

In [3]:
load_dotenv()

# path to save figures
output_path = os.getenv("VIDEO_OUT")

In [52]:
load_path = os.path.join(output_path, 'video_data_time_series_reduced.csv')
df = pd.read_csv(load_path)

In [53]:
df

,filename,frame,video_id,emotion_1_id,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,A227_neg_sur_v_4,1,A227,11,0.000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.088643,0.000,0.144,0.614610,0.0
1,A227_neg_sur_v_4,2,A227,11,0.000,0.000000,0.000000,0.077135,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.063712,0.012,0.164,0.526448,0.0
2,A227_neg_sur_v_4,3,A227,11,0.000,0.000000,0.000000,0.129477,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000,0.000000,0.077562,0.020,0.186,0.483627,0.0
3,A227_neg_sur_v_4,4,A227,11,0.000,0.000000,0.000000,0.165289,0.000000,0.010,...,0.000000,0.000000,0.000000,0.000,0.000000,0.099723,0.018,0.214,0.450882,0.0
4,A227_neg_sur_v_4,5,A227,11,0.000,0.000000,0.000000,0.099174,0.000000,0.010,...,0.000000,0.000000,0.000000,0.000,0.000000,0.110803,0.018,0.232,0.471033,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70321,A303_nos_v_4,179,A303,30,0.026,0.000000,0.846939,0.005510,0.183932,0.482,...,0.235669,0.106818,0.357303,0.000,0.037113,0.000000,0.000,0.030,0.030227,0.0
70322,A303_nos_v_4,180,A303,30,0.026,0.000000,0.842857,0.005510,0.177590,0.458,...,0.237792,0.106818,0.366292,0.000,0.039175,0.000000,0.000,0.024,0.042821,0.0
70323,A303_nos_v_4,181,A303,30,0.018,0.000000,0.818367,0.000000,0.181818,0.436,...,0.231423,0.097727,0.370787,0.002,0.055670,0.000000,0.000,0.014,0.075567,0.0
70324,A303_nos_v_4,182,A303,30,0.036,0.020121,0.816327,0.000000,0.183932,0.422,...,0.252654,0.097727,0.393258,0.002,0.061856,0.000000,0.000,0.014,0.078086,0.0


In [54]:
AU17 = df[["filename", "emotion_1_id", "AU17_r"]]

In [55]:
AU17.head(200)

,filename,emotion_1_id,AU17_r
0,A227_neg_sur_v_4,11,0.000000
1,A227_neg_sur_v_4,11,0.000000
2,A227_neg_sur_v_4,11,0.000000
3,A227_neg_sur_v_4,11,0.000000
4,A227_neg_sur_v_4,11,0.000000
...,...,...,...
195,A227_neg_sur_v_4,11,0.090722
196,A227_neg_sur_v_4,11,0.082474
197,A227_neg_sur_v_4,11,0.072165
198,A227_neg_sur_v_4,11,0.053608


In [98]:
X = []
y = []

In [99]:
for _, group in AU17.groupby('filename'):
    x_list = list(group["AU17_r"])
    X.append(x_list)
    
    y_list = group["emotion_1_id"].unique()
    if len(y_list) == 1:
        y.append(y_list[0])
    else:
        raise ValueError('More than 1 label for time series encountered')

In [100]:
y = np.asarray(y)

In [101]:
length = max(map(len, X))
X=np.array([xi+[0]*(length-len(xi)) for xi in X])

In [103]:
X.shape

(298, 800)

In [104]:
y.shape

(298,)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [111]:
clf = TimeSeriesForest(random_state=43)
clf.fit(X_train, y_train)

ret = clf.score(X_test, y_test)

In [113]:
ret

0.06666666666666667